# Check GPU availability
Check GPU drivers and CUDA availability for training.

In [ ]:
# Check GPU status
!nvidia-smi

# Install required packages
Install required packages for downloading dataset and training YOLOv11 model.

In [ ]:
# Install required packages (without comet_ml)
print("Installing required packages...")
%pip install -q ultralytics roboflow matplotlib seaborn pandas
print("Packages installed successfully!")

In [ ]:
# Import necessary libraries
import os
from google.colab import userdata

# Roboflow configuration
ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
ROBOFLOW_WORKSPACE = userdata.get('ROBOFLOW_WORKSPACE')
ROBOFLOW_PROJECT = userdata.get('ROBOFLOW_PROJECT')
ROBOFLOW_VERSION = userdata.get('ROBOFLOW_VERSION')

print("[OK] Configuration loaded successfully!")

# Download dataset from Roboflow
Download dataset from Roboflow for YOLOv11 training.

In [ ]:
# Download dataset
from roboflow import Roboflow

print("Downloading dataset from Roboflow...")
print(f"Workspace: {ROBOFLOW_WORKSPACE}")
print(f"Project: {ROBOFLOW_PROJECT}")
print(f"Version: {ROBOFLOW_VERSION}")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(int(ROBOFLOW_VERSION))
dataset = version.download("yolov11")

print("Dataset downloaded successfully!")

# Model Evaluation - Single Model

Evaluate a single YOLOv11 segmentation model on the test dataset.
This section will:
1. Load the model
2. Run validation on the test set
3. Collect and display all metrics
4. Visualize results with charts

In [ ]:
# Evaluate single model and collect metrics
import os
import pandas as pd
from ultralytics import YOLO

# Define model path (change this to evaluate different models)
MODEL_PATH = "/content/yolo11n-seg-version-1-0-0.pt"

# Get data.yaml path from downloaded dataset
data_yaml = f"{dataset.location}/data.yaml"

# Get model name
model_name = os.path.basename(MODEL_PATH).replace(".pt", "")

print("=" * 70)
print("EVALUATING MODEL ON TEST DATASET")
print("=" * 70)
print(f"Model: {model_name}")
print(f"Path: {MODEL_PATH}")
print("-" * 70)

# Load model
model = YOLO(MODEL_PATH)

# Run validation on test set
metrics = model.val(data=data_yaml, split='test', verbose=False)

# Collect metrics into a dictionary
metrics_dict = {
    # Box metrics
    'Box_mAP50-95': metrics.box.map,
    'Box_mAP50': metrics.box.map50,
    'Box_mAP75': metrics.box.map75,
    'Box_Precision': metrics.box.mp,
    'Box_Recall': metrics.box.mr,
    # Segmentation mask metrics
    'Mask_mAP50-95': metrics.seg.map,
    'Mask_mAP50': metrics.seg.map50,
    'Mask_mAP75': metrics.seg.map75,
    'Mask_Precision': metrics.seg.mp,
    'Mask_Recall': metrics.seg.mr,
}

# Create DataFrame for display
results_df = pd.DataFrame([metrics_dict], index=[model_name])

print("\n" + "=" * 70)
print("EVALUATION RESULTS")
print("=" * 70)

# Print Box metrics
print("\nBOUNDING BOX METRICS:")
print("-" * 40)
print(f"  mAP50-95:  {metrics.box.map:.4f}")
print(f"  mAP50:     {metrics.box.map50:.4f}")
print(f"  mAP75:     {metrics.box.map75:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall:    {metrics.box.mr:.4f}")

# Print Mask metrics
print("\nSEGMENTATION MASK METRICS:")
print("-" * 40)
print(f"  mAP50-95:  {metrics.seg.map:.4f}")
print(f"  mAP50:     {metrics.seg.map50:.4f}")
print(f"  mAP75:     {metrics.seg.map75:.4f}")
print(f"  Precision: {metrics.seg.mp:.4f}")
print(f"  Recall:    {metrics.seg.mr:.4f}")

print("\n" + "=" * 70)
print("[OK] Model evaluation completed!")

# Visualization - Metrics Charts

Visualize all evaluation metrics using bar charts for the single model.

In [ ]:
# Visualization - Metrics Charts for Single Model
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# ============================================================
# CONFIGURATION - Adjust these values as needed
# ============================================================
Y_AXIS_MIN = 0.9      # Minimum value for Y-axis (bar charts)
Y_AXIS_MAX = 1.0     # Maximum value for Y-axis (bar charts)
RADAR_MIN = 0.9       # Minimum value for Radar chart
RADAR_MAX = 1.0       # Maximum value for Radar chart
# ============================================================

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'YOLOv11 Segmentation Model Evaluation: {model_name}', fontsize=16, fontweight='bold')

# Color palettes
box_colors = ['#3498db', '#2ecc71', '#9b59b6', '#e67e22', '#1abc9c']
mask_colors = ['#e74c3c', '#f39c12', '#8e44ad', '#16a085', '#d35400']

# Colors for Box vs Mask comparison (softer colors)
box_compare_color = '#5dade2'  # Soft blue
mask_compare_color = '#f5b041'  # Soft orange/amber

# 1. Box Metrics Bar Chart
ax1 = axes[0, 0]
box_metrics = ['mAP50-95', 'mAP50', 'mAP75', 'Precision', 'Recall']
box_values = [metrics.box.map, metrics.box.map50, metrics.box.map75, metrics.box.mp, metrics.box.mr]

bars1 = ax1.bar(box_metrics, box_values, color=box_colors, edgecolor='black', linewidth=0.5)
ax1.set_ylabel('Score')
ax1.set_title('Bounding Box Metrics', fontsize=12, fontweight='bold')
ax1.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

# Add value labels
for bar, val in zip(bars1, box_values):
    height = bar.get_height()
    ax1.annotate(f'{val:.3f}',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=10, fontweight='bold')

# 2. Mask Metrics Bar Chart
ax2 = axes[0, 1]
mask_metrics = ['mAP50-95', 'mAP50', 'mAP75', 'Precision', 'Recall']
mask_values = [metrics.seg.map, metrics.seg.map50, metrics.seg.map75, metrics.seg.mp, metrics.seg.mr]

bars2 = ax2.bar(mask_metrics, mask_values, color=mask_colors, edgecolor='black', linewidth=0.5)
ax2.set_ylabel('Score')
ax2.set_title('Segmentation Mask Metrics', fontsize=12, fontweight='bold')
ax2.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

# Add value labels
for bar, val in zip(bars2, mask_values):
    height = bar.get_height()
    ax2.annotate(f'{val:.3f}',
                 xy=(bar.get_x() + bar.get_width()/2, height),
                 xytext=(0, 3),
                 textcoords="offset points",
                 ha='center', va='bottom',
                 fontsize=10, fontweight='bold')

# 3. Box vs Mask Comparison
ax3 = axes[1, 0]
x = np.arange(len(box_metrics))
width = 0.35

bars_box = ax3.bar(x - width/2, box_values, width, label='Box', color=box_compare_color, edgecolor='black', linewidth=0.5)
bars_mask = ax3.bar(x + width/2, mask_values, width, label='Mask', color=mask_compare_color, edgecolor='black', linewidth=0.5)

ax3.set_ylabel('Score')
ax3.set_title('Box vs Mask Metrics Comparison', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels(box_metrics)
ax3.legend(loc='upper left', bbox_to_anchor=(0.02, 0.98), framealpha=0.9)
ax3.set_ylim(Y_AXIS_MIN, Y_AXIS_MAX)

# 4. Radar/Spider Chart
ax4 = axes[1, 1]

# Prepare data for radar chart
categories = ['mAP50-95', 'mAP50', 'mAP75', 'Precision', 'Recall']
N = len(categories)

# Compute angle for each category
angles = [n / float(N) * 2 * np.pi for n in range(N)]
angles += angles[:1]  # Complete the loop

# Box and Mask values
box_vals = box_values + [box_values[0]]
mask_vals = mask_values + [mask_values[0]]

# Create radar chart
ax4 = plt.subplot(2, 2, 4, polar=True)
ax4.set_theta_offset(np.pi / 2)
ax4.set_theta_direction(-1)

# Draw axes with dynamic ticks based on RADAR_MIN
radar_ticks = np.linspace(RADAR_MIN, RADAR_MAX, 5)
radar_labels = [f"{t:.2f}" for t in radar_ticks]
plt.xticks(angles[:-1], categories, fontsize=10)
ax4.set_rlabel_position(0)
plt.yticks(radar_ticks, radar_labels, color="grey", size=8)
plt.ylim(RADAR_MIN, RADAR_MAX)

# Plot data with softer colors
ax4.plot(angles, box_vals, 'o-', linewidth=2, label='Box', color=box_compare_color)
ax4.fill(angles, box_vals, alpha=0.25, color=box_compare_color)
ax4.plot(angles, mask_vals, 'o-', linewidth=2, label='Mask', color=mask_compare_color)
ax4.fill(angles, mask_vals, alpha=0.25, color=mask_compare_color)
ax4.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
ax4.set_title('Metrics Radar Chart', fontsize=12, fontweight='bold', y=1.08)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig(f'{model_name}_evaluation.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n[OK] Charts saved to '{model_name}_evaluation.png'")